In [1]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

In [11]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()


In [23]:
trigram = [((test_sentence[i],test_sentence[i+1]),(test_sentence[i+2]))
          for i in range(len(test_sentence) - 2)]

In [18]:
vocb = set(test_sentence)
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

In [19]:
class NgramModel(nn.Module):
    def __init__(self, vocb_size, context_size, n_dim):
        super(NgramModel, self).__init__()
        self.n_word = vocb_size
        self.embedding = nn.Embedding(self.n_word, n_dim)
        self.linear1 = nn.Linear(context_size * n_dim, 128)
        self.linear2 = nn.Linear(128, self.n_word)

    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.view(1, -1)
        out = self.linear1(emb)
        out = F.relu(out)
        out = self.linear2(out)
        log_prob = F.log_softmax(out)
        return log_prob


ngrammodel = NgramModel(len(word_to_idx), CONTEXT_SIZE, 100)
criterion = nn.NLLLoss()
optimizer = optim.SGD(ngrammodel.parameters(), lr=1e-3)


In [24]:
for epoch in range(100):
    print('epoch: {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0
    for data in trigram:
        word, label = data
        word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
        label = Variable(torch.LongTensor([word_to_idx[label]]))
        # forward
        out = ngrammodel(word)
        loss = criterion(out, label)
        running_loss += loss.data[0]
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Loss: {:.6f}'.format(running_loss / len(word_to_idx)))


epoch: 1
**********
Loss: 5.329770
epoch: 2
**********
Loss: 5.271808
epoch: 3
**********
Loss: 5.214428
epoch: 4
**********
Loss: 5.157653
epoch: 5
**********
Loss: 5.101325
epoch: 6
**********
Loss: 5.045189
epoch: 7
**********
Loss: 4.989182
epoch: 8
**********
Loss: 4.933473
epoch: 9
**********
Loss: 4.877858
epoch: 10
**********
Loss: 4.822221
epoch: 11
**********
Loss: 4.766770
epoch: 12
**********
Loss: 4.711290
epoch: 13
**********
Loss: 4.655749
epoch: 14
**********
Loss: 4.599836
epoch: 15
**********
Loss: 4.543696
epoch: 16
**********
Loss: 4.487429
epoch: 17
**********
Loss: 4.430806
epoch: 18
**********
Loss: 4.373873
epoch: 19
**********
Loss: 4.316477
epoch: 20
**********
Loss: 4.258440
epoch: 21
**********
Loss: 4.200029
epoch: 22
**********
Loss: 4.141170
epoch: 23
**********
Loss: 4.081846
epoch: 24
**********
Loss: 4.021859
epoch: 25
**********
Loss: 3.961349
epoch: 26
**********
Loss: 3.900334
epoch: 27
**********
Loss: 3.838820
epoch: 28
**********
Loss: 3.776951
e

In [43]:
trigram[3]

(('shall', 'besiege'), 'thy')

In [50]:
word

Variable containing:
 72
 83
[torch.LongTensor of size 2]

In [79]:
word, label = trigram[5]
word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
out = ngrammodel(word)
_, predict_label = torch.max(out, 1)
predict_word = idx_to_word[predict_label.data[0][0]]
print('real word is {}, predict word is {}'.format(label, predict_word))

real word is And, predict word is And
